In [38]:
import pandas as pd
import numpy as np
from glob import glob
import folium
from datetime import datetime
from geopy.geocoders import Nominatim

In [39]:
def get_lat_lon(location_name):
    geolocator = Nominatim(user_agent="location_finder")
    location = geolocator.geocode(location_name)
    if location:
        latitude, longitude = location.latitude, location.longitude
        return latitude, longitude
    else:
        return None
     
def generate_map(coordinates, location_name):
    if coordinates:
        lat = coordinates[0]
        long = coordinates[1]
        map = folium.Map(location=[lat, long], zoom_start=150)

        # create an iframe pop-up for the marker
        popup_html = f"<b>Date:</b> 10th October, 2023<br/>"
        popup_html += f"<b>Location:</b> Department of Maths <br/>"
        popup_html += f"<b>Time:</b> {datetime.now()}<br/>"
        popup_html += '<b><a href="{}" target="_blank">Event Page</a></b>'.format(
            "Testing Map"
        )
        popup_iframe = folium.IFrame(width=200, height=110, html=popup_html)

        folium.Marker(
            location=[lat, long],
            popup=folium.Popup(popup_iframe),
            icon=folium.Icon(color="red"),
        ).add_to(map)
        points = [(24.94411, 67.08085), (24.9443, 67.0793), (24.94351, 67.08298)]
        folium.PolyLine(points, color="red", weight=5, opacity=0.85).add_to(map)

        map.save("map.html")
    else:
        print(
            f"Location not found for {location_name}. Please check the spelling or provide more details."
        )

In [40]:
files = glob('./csv_data/*.csv')

In [41]:
files

['./csv_data\\20240125-110332 - Right portion from parking.csv',
 './csv_data\\20240125-111600 - From gate no 2.csv',
 './csv_data\\20240125-112847 - Parking to main library.csv',
 './csv_data\\20240125-114435 - Parking to canteen.csv',
 './csv_data\\20240125-115555 - Main gate to main building.csv']

In [43]:
files[1]

'./csv_data\\20240125-111600 - From gate no 2.csv'

In [44]:
data = pd.read_csv(files[1])

In [45]:
data

,type,date time,latitude,longitude,accuracy(m),altitude(m),geoid_height(m),speed(m/s),bearing(deg),sat_used,sat_inview,name,desc
0,W,2024-01-25 06:16:04,24.850467,67.003802,2.0,5.9,NaN,0.000,NaN,17,41,NaN,Gate no 2
1,W,2024-01-25 06:18:59,24.850852,67.003897,3.4,11.7,NaN,0.000,NaN,16,42,NaN,Media department
2,W,2024-01-25 06:20:29,24.850457,67.004143,1.8,18.9,NaN,0.825,168.0,17,42,NaN,Thomas henry lawn
3,T,2024-01-25 06:16:00,24.850462,67.003793,2.0,5.3,NaN,0.501,349.0,17,41,20240125-111600,GPS Logger: 20240125-111600 - From gate no 2
4,T,2024-01-25 06:16:01,24.850463,67.003793,2.0,5.4,NaN,0.432,344.0,17,41,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,T,2024-01-25 06:20:36,24.850483,67.004123,1.1,18.5,NaN,0.000,NaN,17,42,NaN,NaN
268,T,2024-01-25 06:20:37,24.850485,67.004123,1.3,18.5,NaN,0.000,NaN,17,42,NaN,NaN
269,T,2024-01-25 06:20:38,24.850487,67.004123,1.4,18.5,NaN,0.000,NaN,17,42,NaN,NaN
270,T,2024-01-25 06:20:39,24.850487,67.004123,1.5,18.5,NaN,0.000,NaN,17,42,NaN,NaN


# Removing Extra Columns

In [46]:
data = data[['type','latitude','longitude','desc']]

In [47]:
data

,type,latitude,longitude,desc
0,W,24.850467,67.003802,Gate no 2
1,W,24.850852,67.003897,Media department
2,W,24.850457,67.004143,Thomas henry lawn
3,T,24.850462,67.003793,GPS Logger: 20240125-111600 - From gate no 2
4,T,24.850463,67.003793,NaN
...,...,...,...,...
267,T,24.850483,67.004123,NaN
268,T,24.850485,67.004123,NaN
269,T,24.850487,67.004123,NaN
270,T,24.850487,67.004123,NaN


# Separating Look Up Table

In [48]:
reference = data[data['type'] =='W']

In [53]:
reference

,type,latitude,longitude,desc
0,W,24.850467,67.003802,Gate no 2
1,W,24.850852,67.003897,Media department
2,W,24.850457,67.004143,Thomas henry lawn


# Separating Tracking

In [54]:
tracking = data[data['type'] =='T']

# Marking Land Marks

In [55]:
for lat,long,label in zip(reference.latitude,reference.longitude,reference.desc):
    condition = (tracking['latitude']==lat)&(tracking['longitude']==long)
    print("Marking",label)
    tracking.loc[condition, 'desc'] = label

Marking Gate no 2
Marking Media department
Marking Thomas henry lawn


In [56]:
lat,long = get_lat_lon('Sindh Madressa-tul-Islam University')
smiu_map = folium.Map(location=[lat, long], zoom_start=150)

In [57]:
def fetch_tracking_records(df, start_location, end_location):
    start_location_df = df[df['desc'] == start_location]
    end_location_df = df[df['desc'] == end_location]
    if start_location_df.empty or end_location_df.empty:
        print(f"One or both of the specified locations not found.")
        return pd.DataFrame()
    start_index = start_location_df.index[0]
    end_index = end_location_df.index[0]
    tracking_records = df.loc[start_index:end_index, ['latitude', 'longitude']]
    return tracking_records

In [58]:
# locations = folium.map.FeatureGroup()

# for lat, lng in zip(reference.latitude, reference.longitude):
#     locations.add_child(
#         folium.CircleMarker(
#             [lat, lng],
#             radius=15,
#             color='yellow',
#             fill=True,
#             fill_color='blue',
#             fill_opacity=0.6
#         )
#     )

for lat, lng, label in zip(reference.latitude, reference.longitude, reference.desc):
    folium.Marker(
        [lat, lng],
        popup=label,
    ).add_to(smiu_map)

smiu_map

In [63]:
try:
    start_location = "Media department"
    end_location = "Gate no 2"
    tracking_records = fetch_tracking_records(tracking, start_location, end_location)
    points = [(x, y) for x, y in zip(tracking_records.latitude, tracking_records.longitude)]
    folium.PolyLine(points, color="red", weight=5, opacity=0.85).add_to(smiu_map)
except:
    temp = start_location
    start_location = end_location
    end_location = temp
    tracking_records = fetch_tracking_records(tracking, start_location, end_location)
    points = [(x, y) for x, y in zip(tracking_records.latitude, tracking_records.longitude)]
    folium.PolyLine(points, color="red", weight=5, opacity=0.85).add_to(smiu_map)

In [64]:
smiu_map